In [1]:
import time
import astroquery.heasarc
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from collections import defaultdict
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.io import ascii

Jupiter's monthly positions

In [2]:
epochs = {'start':'2003-01-01', 'stop':'2025-01-01', 'step':'31d'}

jupiter = Horizons(id='599', location='@0',epochs=epochs) 
eph = jupiter.ephemerides()

ra_values = eph['RA']
dec_values = eph['DEC']
datetime_values = eph['datetime_jd']
print(f"Jupiter's monthly position between {epochs['start']} and {epochs['stop']}: \n")
print(eph[['datetime_str', 'RA', 'DEC']])

monthly_intervals = []
for i in range(len(datetime_values) - 1):  
    start_time = Time(datetime_values[i], format='jd').mjd
    end_time = Time(datetime_values[i + 1], format='jd').mjd
    monthly_intervals.append((start_time, end_time))


Jupiter's monthly position between 2003-01-01 and 2025-01-01: 

   datetime_str       RA      DEC   
       ---           deg      deg   
----------------- --------- --------
2003-Jan-01 00:00 133.13461 18.23402
2003-Feb-01 00:00 135.64353 17.59249
2003-Mar-04 00:00  138.1248 16.92216
2003-Apr-04 00:00 140.57899 16.22478
2003-May-05 00:00 143.00678 15.50206
2003-Jun-05 00:00 145.40891  14.7557
2003-Jul-06 00:00 147.78632 13.98736
2003-Aug-06 00:00    150.14 13.19866
2003-Sep-06 00:00 152.47101 12.39117
2003-Oct-07 00:00 154.78053 11.56645
              ...       ...      ...
2024-Mar-21 00:00  50.60927 17.52697
2024-Apr-21 00:00  53.43087 18.24997
2024-May-22 00:00  56.26678 18.92932
2024-Jun-22 00:00    59.116 19.56331
2024-Jul-23 00:00  61.97734 20.15039
2024-Aug-23 00:00  64.84944 20.68914
2024-Sep-23 00:00  67.73074 21.17832
2024-Oct-24 00:00  70.61942 21.61685
2024-Nov-24 00:00  73.51359 22.00384
2024-Dec-25 00:00  76.41117 22.33857
Length = 260 rows


Get the entire full sky table

In [3]:
Heasarc = astroquery.heasarc.Heasarc()

load = True
if load:
    table = ascii.read('../data/full_table.dat')
else:
    table = Heasarc.query_region(SkyCoord(ra=0, dec=0, unit='deg'), radius=90*u.deg, maxrec=1000000, catalog='intscw')
    ascii.write(table, '../data/full_table.dat', overwrite=True)

table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64
207800280021,1,slew,public,Y,47.95158,-89.85108,975,976,0,975,974,976,58584.8798067897,58584.8911030961,--,--,--,42106880,1
218500510010,1,pointing,public,Y,294.97232,-89.65222,5772,5770,1709,5772,5772,5772,58870.8615711961,58870.9290133643,17200120001,General,Moritz M. M. Pleintinger,250466304,3
218300230021,1,slew,public,Y,27.95042,-89.55722,137,137,0,135,135,0,58864.2218220474,58864.2234076965,17200120001,General,Moritz M. M. Pleintinger,6930432,7
230300440021,1,slew,public,Y,58.73615,-89.51254,136,137,0,136,136,0,59184.3325168109,59184.3341024602,--,--,--,7454720,11
229900030021,1,slew,public,Y,275.12358,-89.43092,115,110,0,115,81,0,59172.0166874964,59172.0180300899,17200120002,General,Moritz M. M. Pleintinger,6631424,14
230300490010,1,pointing,public,Y,11.65258,-89.23714,3472,3084,3440,3472,3472,3472,59184.572320224,59184.6131188626,17200120002,General,Moritz M. M. Pleintinger,159584256,23
218400210010,1,pointing,public,Y,284.91614,-89.19434,3504,3502,3472,3504,3504,3503,58866.860677714,58866.9015342187,17200120001,General,Moritz M. M. Pleintinger,154615808,25
229200810021,1,slew,public,Y,275.15002,-89.16277,133,134,0,132,124,0,59155.4036450234,59155.4051959502,17200120002,General,Moritz M. M. Pleintinger,7000064,27
218400520010,1,pointing,public,Y,351.69272,-89.14411,3484,3484,3467,3484,3484,3484,58868.210528019,58868.2514308201,17200120001,General,Moritz M. M. Pleintinger,156672000,30


Filter for pointing type

In [4]:
pointing_table = table[table['scw_type'] == 'pointing']
pointing_table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64
218500510010,1,pointing,public,Y,294.97232,-89.65222,5772,5770,1709,5772,5772,5772,58870.8615711961,58870.9290133643,17200120001,General,Moritz M. M. Pleintinger,250466304,3
230300490010,1,pointing,public,Y,11.65258,-89.23714,3472,3084,3440,3472,3472,3472,59184.572320224,59184.6131188626,17200120002,General,Moritz M. M. Pleintinger,159584256,23
218400210010,1,pointing,public,Y,284.91614,-89.19434,3504,3502,3472,3504,3504,3503,58866.860677714,58866.9015342187,17200120001,General,Moritz M. M. Pleintinger,154615808,25
218400520010,1,pointing,public,Y,351.69272,-89.14411,3484,3484,3467,3484,3484,3484,58868.210528019,58868.2514308201,17200120001,General,Moritz M. M. Pleintinger,156672000,30
218400520020,1,pointing,public,Y,351.69272,-89.14411,4511,4510,0,3352,3351,4512,58868.2514308201,58868.303653072,17200120001,General,Moritz M. M. Pleintinger,209494016,31
218400020010,1,pointing,public,Y,62.20154,-89.06347,2368,2300,0,1962,1962,1252,58866.0568462387,58866.0984319108,17200120001,General,Moritz M. M. Pleintinger,160223232,32
218400010010,1,pointing,public,Y,62.29213,-89.06245,643,0,0,0,0,1054,58866.0422281748,58866.0555036454,--,--,--,13348864,34
218500190010,1,pointing,public,Y,24.07567,-89.05664,3487,3485,3458,3487,3487,3484,58869.4612579008,58869.5019986645,17200120001,General,Moritz M. M. Pleintinger,154918912,35
229900100010,1,pointing,public,Y,289.3252,-89.00825,3478,3218,3443,3478,3478,3478,59172.2876946125,59172.3285163969,17200120002,General,Moritz M. M. Pleintinger,157827072,36


Filter for good_isgri > 1000

In [5]:
good_isgri_table = pointing_table[pointing_table['good_isgri'] > 1000]
good_isgri_table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64
218500510010,1,pointing,public,Y,294.97232,-89.65222,5772,5770,1709,5772,5772,5772,58870.8615711961,58870.9290133643,17200120001,General,Moritz M. M. Pleintinger,250466304,3
230300490010,1,pointing,public,Y,11.65258,-89.23714,3472,3084,3440,3472,3472,3472,59184.572320224,59184.6131188626,17200120002,General,Moritz M. M. Pleintinger,159584256,23
218400210010,1,pointing,public,Y,284.91614,-89.19434,3504,3502,3472,3504,3504,3503,58866.860677714,58866.9015342187,17200120001,General,Moritz M. M. Pleintinger,154615808,25
218400520010,1,pointing,public,Y,351.69272,-89.14411,3484,3484,3467,3484,3484,3484,58868.210528019,58868.2514308201,17200120001,General,Moritz M. M. Pleintinger,156672000,30
218400520020,1,pointing,public,Y,351.69272,-89.14411,4511,4510,0,3352,3351,4512,58868.2514308201,58868.303653072,17200120001,General,Moritz M. M. Pleintinger,209494016,31
218400020010,1,pointing,public,Y,62.20154,-89.06347,2368,2300,0,1962,1962,1252,58866.0568462387,58866.0984319108,17200120001,General,Moritz M. M. Pleintinger,160223232,32
218500190010,1,pointing,public,Y,24.07567,-89.05664,3487,3485,3458,3487,3487,3484,58869.4612579008,58869.5019986645,17200120001,General,Moritz M. M. Pleintinger,154918912,35
229900100010,1,pointing,public,Y,289.3252,-89.00825,3478,3218,3443,3478,3478,3478,59172.2876946125,59172.3285163969,17200120002,General,Moritz M. M. Pleintinger,157827072,36
229400830010,1,pointing,public,Y,9.95142,-89.0055,4977,3116,0,1578,1578,4163,59160.8586945797,59160.9184747059,17200120002,General,Moritz M. M. Pleintinger,191270912,37


Jupiter table

In [6]:
radius = 8

temp_table = good_isgri_table.copy()
table_coords = SkyCoord(ra=temp_table['ra'], dec=temp_table['dec'], unit='deg')

mask = np.zeros(len(temp_table), dtype=bool)

for ra, dec, month in zip(ra_values, dec_values, monthly_intervals):
    jupiter_coords = SkyCoord(ra=ra, dec=dec, unit='deg')
    sep = table_coords.separation(jupiter_coords).deg
    mask |= (sep <= 8) & (month[0] <= temp_table['start_date']) & (temp_table['start_date'] <= month[1])

jupiter_table = temp_table[mask]
jupiter_table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64
214500490010,1,pointing,public,Y,270.52011,-30.52128,0,3522,3491,3514,3514,3523,58763.8205709193,58763.86172835,16200280002,General,Dr. Sergei Grebenev,90554368,141240
215300220010,1,pointing,public,Y,270.38593,-30.489,1933,1925,1906,1931,1931,1933,58784.0822160318,58784.105306325,16200230026,General,Prof. Joern Wilms,90542080,141406
215300190010,1,pointing,public,Y,272.86359,-30.16489,1986,1979,1953,1955,1955,1968,58784.0084312594,58784.0315794234,16200230026,General,Prof. Joern Wilms,91574272,143684
214400770010,1,pointing,public,Y,271.25949,-29.80186,0,1945,1921,1946,1946,1946,58761.8276646893,58761.8508244246,16200230027,General,Prof. Joern Wilms,50688000,147557
66300520010,1,pointing,public,Y,272.51428,-29.62525,3172,3170,3148,3172,0,3172,54545.3200781411,54545.3574624228,05300000002,Key Programme,Key Programme,147229696,148875
214500560010,1,pointing,public,Y,270.50864,-29.55881,0,3496,3466,3497,3497,3497,58764.1179553466,58764.1591243514,16200280002,General,Dr. Sergei Grebenev,92680192,149420
215200220010,1,pointing,public,Y,271.32175,-29.26306,1944,1942,1917,1944,1933,1944,58781.4243671216,58781.4475268594,16200230037,General,Prof. Joern Wilms,91422720,151461
215200190010,1,pointing,public,Y,273.77643,-28.89775,1923,1921,1901,1923,1923,1923,58781.3506517934,58781.3736726427,16200230037,General,Prof. Joern Wilms,90025984,155243
220800140010,1,pointing,public,Y,277.24072,-28.7365,0,1944,1922,1947,1947,1947,58930.1338896352,58930.1570493713,17200180006,General,Prof. Joern Wilms,52928512,156684


Sort table

In [7]:
jupiter_table.sort('start_date')
jupiter_table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64
66100480010,1,pointing,public,Y,270.67905,-27.46044,3196,3187,3183,3053,0,3196,54539.2077828315,54539.2451671126,05300000002,Key Programme,Key Programme,149592064,166613
66100490010,1,pointing,public,Y,270.12726,-25.34595,3174,3171,3154,3047,0,3174,54539.246509706,54539.2838939872,05300000002,Key Programme,Key Programme,151291904,177308
66100510010,1,pointing,public,Y,270.16653,-23.16842,3224,3221,3191,2813,0,3198,54539.3236625289,54539.36104681,05300000002,Key Programme,Key Programme,149195776,184678
66100520010,1,pointing,public,Y,270.74258,-25.27389,3226,3223,3191,3140,0,3200,54539.3625630146,54539.3999588697,05300000002,Key Programme,Key Programme,150768640,177579
66100530010,1,pointing,public,Y,271.35147,-27.37131,3220,3219,3203,3072,0,3202,54539.401289889,54539.4386857441,05300000002,Key Programme,Key Programme,149855232,167084
66300470010,1,pointing,public,Y,270.08972,-26.18747,3170,3168,3148,3052,0,3170,54545.1241289485,54545.1615132305,05300000002,Key Programme,Key Programme,149064704,174014
66300490010,1,pointing,public,Y,271.3656,-23.20119,3038,3038,3038,3038,0,3038,54545.203700756,54545.2411081861,05300000002,Key Programme,Key Programme,148412416,184607
66300500010,1,pointing,public,Y,271.72787,-25.34639,3230,3227,3191,3143,0,3197,54545.2426243907,54545.2800202466,05300000002,Key Programme,Key Programme,148937728,177305
66300510010,1,pointing,public,Y,272.10693,-27.48767,3203,3201,3180,2968,0,3199,54545.2813512659,54545.3187471218,05300000002,Key Programme,Key Programme,148806656,166437


Save to file

In [8]:
ascii.write(jupiter_table, '../data/jupiter_table.dat', overwrite=True)
# jupiter_table = ascii.read('../data/jupiter_table.dat')

Crab table

In [9]:
radius = 8.

crab_coords = SkyCoord.from_name('Crab')
crab_mask = (crab_coords.separation(table_coords).deg <= radius)

crab_table = temp_table[crab_mask]
crab_table

scw_id,scw_ver,scw_type,status,data_in_heasarc,ra,dec,good_spi,good_isgri,good_picsit,good_jemx1,good_jemx2,good_omc,start_date,end_date,obs_id,obs_type,pi_name,data_size,__row
int64,int64,str8,str7,str1,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,str62,str13,str93,int64,int64


Sort

In [10]:
crab_table.sort('start_date')

Save to file

In [11]:
ascii.write(crab_table, '../data/crab_table.dat', overwrite=True)
# crab_table = ascii.read('../data/crab_table.dat')